<h1> NEAT Implementation Practice </h1>
A first-pass attempt at implementing NEAT by translating SethBling's neatevolve.lua script into Python.

In [39]:
import gym
import numpy as np # might not even need this ?
import random # pseudo-random|
import math

In [40]:
# first define the environment and get the both the action and observation spaces 
envName = 'CartPole-v0' # just need to change this if you wanna try it in environments
env = gym.make(envName)

# print out the observation and action spaces
print(env.action_space.n) # number of valid actions we can take 
print(env.observation_space.shape[0]) # number of "sensors" of the environment that we have (number of inputs)

# NOTE: Need to keep track of the valid values these can take 
# NOTE: Also need to have inputs + 1, with the 1 being the bias (stc?)
outputs = env.action_space.n
inputs = env.observation_space.shape[0] + 1



2
4


In [41]:
# hyperparameters, according to section 4.1 Parameter Settings
deltaThreshold = 3.0

c1 = c2 = 1.0
c3 = 0.4
perturbVal = 0.05


numUntilStagnant = 15


genomeMutationChance = 0.8
connectionUniformPerturb = 0.9
connectionRandomValue = 1 - connectionUniformPerturb
disableGeneChance = 0.75
mutateNoCrossoverChance = 0.25
interspeciesMatingRate = 0.001
smallPopulationNewNodeChance = 0.03
smallPopulationNewConnectionChance = 0.05
largePopulationNewConnectionChance = 0.3

In [42]:
def sigmoid(x):
    return 1/(1+np.exp(-x))

def steepsigmoid(x):
    return 1/(1+np.exp(-4.9*x))

In [43]:
x = np.array([1, 2, 3])
w = np.array([0.5, 0.1, 0.3])

In [44]:
val = sigmoid(x.dot(w))
print(val)

0.8320183851339245


In [95]:
class Genome:
    # initialize class variables 
    globalInnovationNumber = 1
    def __init__(self, inputs, outputs):
        self.globalInnovation = 1
        self.nodeGenes = {}
        self.connectGenes = {}
        self.fitness = 0
        self.sharedFitness = 0
        self.species = 0
        self.inputCount = inputs
        self.outputCount = outputs
        self.mutationSuccess = False 
        
    def setGenes(nodeGenes, connectGenes):
        self.nodeGenes = nodeGenes
        self.connectGenes = connectGenes
        
    @staticmethod
    def copyGenome(genome):
        g = Genome()
        g.nodeGenes = genome.nodeGenes
        g.connectGenes = genome.connectGenes
        return g
        
    def clearGenome(self):
        self.globalInnovation = 1
        self.nodeGenes = {}
        self.connectGenes = {}
        
    def clearNodeValues(self):
        for node in self.nodeGenes:
            self.nodeGenes[node][1] = 0.0
            
    def initEmptyGenome(self):
        for i in range(self.inputCount):
            self.insertNode('input')
        for i in range(self.outputCount):
            self.insertNode('output')
            
        
    def initRandomGenome(self):
        self.initEmptyGenome()
        
        # brute force connect each new input node to each output node 
        for node in self.nodeGenes:
            if self.nodeGenes[node][0] == 'input':
                for otherNode in self.nodeGenes:
                    if self.nodeGenes[otherNode][0] == 'output':
                        self.insertConnection(node, otherNode, random.random())
             
    def insertNode(self, nodeType='input', value = 0.0):
        self.nodeGenes[len(self.nodeGenes)+1]=[nodeType, value]
        
    def insertConnection(self, outNode, inNode, weight = 0.5, isExpressed = True):
        if (outNode in self.nodeGenes) and (inNode in self.nodeGenes): # check if the nodes even exist 
            if ((outNode, inNode) not in self.connectGenes and (inNode, outNode) not in self.connectGenes and self.nodeGenes[inNode][0] != 'input' and outNode != inNode): # check that the connection doesnt already exist
                # also dont allow recurrent connections (only feedforward)
                self.connectGenes[(outNode, inNode)] = [self.globalInnovation, weight, isExpressed]
                Genome.globalInnovationNumber += 1
                self.mutationSuccess = True
            else: 
                print('Connection already exists. Did not insert')
        else:
            print('Could not find node(s). Did not insert.')
            
    def checkConnection(self, outNode, inNode):
        if (outNode in self.nodeGenes) and (inNode in self.nodeGenes):
            if (outNode, inNode) in self.connectGenes:
                return self.connectGenes[(outNode, inNode)][2]
            else:
                print('Could not find the connection')
        else:
            print('Could not find node(s). Did not disable.')
            
    def disableConnection(self, outNode, inNode):
        if (outNode in self.nodeGenes) and (inNode in self.nodeGenes):
            if (outNode, inNode) in self.connectGenes:
                self.connectGenes[(outNode, inNode)][2] = False
            else:
                print('Could not find connection to disable. Did not disable.')
        else: 
            print('Could not find node(s). Did not disable.')
    
    def mutateAddConnection(self, outNode, inNode):
        self.insertConnection(outNode, inNode, random.random(), True)
            
    def mutateAddRandomConnection(self):
        self.mutationSuccess = False
        while (not self.mutationSuccess):
            self.mutateAddConnection(random.choice(list(self.nodeGenes)),random.choice(list(self.nodeGenes)))
        self.mutationSuccess = False
        
    def mutateAddNode(self, outNode, inNode):
        if (outNode in self.nodeGenes) and (inNode in self.nodeGenes):
            if (outNode, inNode) in self.connectGenes:
                self.insertNode('hidden', '0.0')
                # disable old connection'
                self.disableConnection(outNode, inNode)
                self.insertConnection(outNode, len(self.nodeGenes), 1.0, True)
                # get original weight 
                originalWeight = self.connectGenes[(outNode, inNode)][1]
                self.insertConnection(len(self.nodeGenes), inNode, originalWeight, True)
                self.mutationSuccess = True
                print('added hidden node')
            else:
                print('Error! No connection found.')
        else:
            print('Could not find node(s). Did not mutate.')
    
    def mutateAddRandomNode(self):
        self.mutationSuccess = False
        while (not self.mutationSuccess):
            randomTuple = random.choice(list(self.connectGenes))
            self.mutateAddNode(randomTuple[0],randomTuple[1])
        self.mutationSuccess = False
    
    # this is so fucking slow
    def evaluateAction(self, observation):
        # TODO: IMPLEMENT THIS
        if not self.connectGenes:
            return env.action_space.sample()
        else:
            self.clearNodeValues()
            self.nodeGenes[1][1] = 1.0
            for i in range(self.inputCount-1):
                # print('Loading value: {} into node input: {}'.format(observation[i], i+2))
                self.nodeGenes[i+2][1] = observation[i]
            
            for node in self.nodeGenes:
                if self.nodeGenes[node][0] == 'hidden': # for every hidden node
                    x = []
                    w = []
                    for connection in self.connectGenes:
                        if connection[1] == node: # if inNode == node
                            x.append(self.nodeGenes[connection[0]][1])
                            w.append(self.connectGenes[connection][1])
                    self.nodeGenes[node][1] = steepsigmoid(np.dot(x, w))
                    print('hidden node #{}\'s value: {}'.format(node, self.nodeGenes[node][1]))
            softmaxsum = 0
            for node in self.nodeGenes:
                if self.nodeGenes[node][0] == 'output':
                    x = []
                    w = []
                    for connection in self.connectGenes:
                        if connection[1] == node:
                            x.append(self.nodeGenes[connection[0]][1])
                            w.append(self.connectGenes[connection][1])
                    self.nodeGenes[node][1] = np.exp(np.dot(x, w))
                    softmaxsum += np.exp(np.dot(x, w))
                    # print('output node #{}\'s unweighted value: {}'.format(node, self.nodeGenes[node][1]))
            output = []
            for node in self.nodeGenes:
                if self.nodeGenes[node][0] == 'output':
                    output.append(self.nodeGenes[node][1] / softmaxsum)
            print(output)
            print('Genome takes action {}'.format(output.index(max(output))))
            return output.index(max(output))
                
    
    def printNodeGenes(self):
        print(self.nodeGenes)
        
    def printConnectGenes(self):
        print(self.connectGenes)
    
    def printConnectGeneCount(self):
        print(len(self.connectGenes))
    
    def printActiveConnectGenes(self):
        for connection in self.connectGenes:
            if (self.connectGenes[connection][2]):
                print(connection, self.connectGenes[connection][1])
        
    def printGlobalInnovation(self):
        print(self.globalInnovation)

In [98]:
inputs= env.observation_space.shape[0] + 1
outputs = env.action_space.n
for tries in range(150):
    observation = env.reset()
    genome=Genome(inputs=inputs, outputs=outputs)
    genome.initRandomGenome()
    genome.mutateAddRandomNode()
    genome.mutateAddRandomNode()
    genome.mutateAddRandomNode()
    genome.mutateAddRandomNode()
    genome.mutateAddRandomConnection()
    genome.mutateAddRandomConnection()
    genome.mutateAddRandomConnection()
    genome.mutateAddRandomConnection()
    genome.mutateAddRandomConnection()
    genome.mutateAddRandomConnection()
    t = 0
    while(True):
        env.render()
        action = genome.evaluateAction(observation)
        observation, reward, done, info = env.step(action)
        genome.fitness += reward
        if done:
            print("Episode finished after {} timesteps".format(t+1))
            print("Genome fitness: {}".format(genome.fitness))
            break
        t += 1
            
env.close()

added hidden node
added hidden node
added hidden node
added hidden node
Connection already exists. Did not insert
Connection already exists. Did not insert
Connection already exists. Did not insert
Connection already exists. Did not insert
Connection already exists. Did not insert
Connection already exists. Did not insert
Connection already exists. Did not insert
Connection already exists. Did not insert
Connection already exists. Did not insert
Connection already exists. Did not insert
Connection already exists. Did not insert
Connection already exists. Did not insert
Connection already exists. Did not insert
Connection already exists. Did not insert
hidden node #8's value: 0.5382392307626369
hidden node #9's value: 0.5500823322842892
hidden node #10's value: 0.5147934766422196
hidden node #11's value: 0.5163912788802915
[0.3074682801880254, 0.6925317198119746]
Genome takes action 1
hidden node #8's value: 0.539181644491364
hidden node #9's value: 0.27344197194104974
hidden node #10's

hidden node #8's value: 0.9932775490624298
hidden node #9's value: 0.9839049053325952
hidden node #10's value: 0.9990180193529339
hidden node #11's value: 0.9925730718162848
[0.5414501449931279, 0.45854985500687195]
Genome takes action 0
hidden node #8's value: 0.9936142306932253
hidden node #9's value: 0.977274283758035
hidden node #10's value: 0.9990186924721765
hidden node #11's value: 0.992573096130435
[0.5523196861788964, 0.44768031382110357]
Genome takes action 0
hidden node #8's value: 0.9940850914784556
hidden node #9's value: 0.9679928241875997
hidden node #10's value: 0.9990196330794338
hidden node #11's value: 0.99257313010655
[0.5644054268570179, 0.43559457314298217]
Genome takes action 0
hidden node #8's value: 0.9946594943972455
hidden node #9's value: 0.9550802672844494
hidden node #10's value: 0.9990207793064192
hidden node #11's value: 0.9925731715097434
[0.5775954509288158, 0.42240454907118424]
Genome takes action 0
hidden node #8's value: 0.99530211528088
hidden node

hidden node #8's value: 0.9926084586557181
hidden node #9's value: 0.9281649647802114
hidden node #10's value: 0.9926084586557181
hidden node #11's value: 0.9897752338534203
[0.5330129917908742, 0.4669870082091258]
Genome takes action 0
hidden node #8's value: 0.9926084586557181
hidden node #9's value: 0.9042211100844699
hidden node #10's value: 0.9926084586557181
hidden node #11's value: 0.9885329433067438
[0.5028155503377918, 0.49718444966220815]
Genome takes action 0
hidden node #8's value: 0.9926084586557181
hidden node #9's value: 0.8760013003619201
hidden node #10's value: 0.9926084586557181
hidden node #11's value: 0.9868635004035282
[0.4725374130347478, 0.5274625869652523]
Genome takes action 1
hidden node #8's value: 0.9926084586557181
hidden node #9's value: 0.8949187354677441
hidden node #10's value: 0.9926084586557181
hidden node #11's value: 0.9880117973049384
[0.5071756994715682, 0.49282430052843174]
Genome takes action 0
Episode finished after 22 timesteps
Genome fitness

hidden node #8's value: 0.024471956126780492
hidden node #9's value: 0.9992200990061281
hidden node #10's value: 0.998138510920408
hidden node #11's value: 0.9926084586557181
[0.6938006506200584, 0.3061993493799417]
Genome takes action 0
hidden node #8's value: 0.009521319865753539
hidden node #9's value: 0.9995686992430444
hidden node #10's value: 0.9987638937554314
hidden node #11's value: 0.9926084586557181
[0.6573607897344232, 0.3426392102655767]
Genome takes action 0
hidden node #8's value: 0.003666528779522584
hidden node #9's value: 0.9997645911656265
hidden node #10's value: 0.9992079373530731
hidden node #11's value: 0.9926084586557181
[0.6195399105302836, 0.3804600894697165]
Genome takes action 0
hidden node #8's value: 0.001405626281522941
hidden node #9's value: 0.9998735196459733
hidden node #10's value: 0.9995112113187595
hidden node #11's value: 0.9926084586557181
[0.5800684240094978, 0.4199315759905022]
Genome takes action 0
hidden node #8's value: 0.00053788539467124
h

hidden node #8's value: 0.6582731889696624
hidden node #9's value: 0.36392852594031816
hidden node #10's value: 0.43785702249409403
hidden node #11's value: 0.9976436798225051
[0.2985599469512892, 0.7014400530487108]
Genome takes action 1
hidden node #8's value: 0.713463813001371
hidden node #9's value: 0.11214536853095347
hidden node #10's value: 0.4231223195177222
hidden node #11's value: 0.998813202810424
[0.22571547404355685, 0.7742845259564431]
Genome takes action 1
hidden node #8's value: 0.7664280010361544
hidden node #9's value: 0.02666977546278559
hidden node #10's value: 0.40124332524169604
hidden node #11's value: 0.9993893099229436
[0.1805930703140039, 0.8194069296859962]
Genome takes action 1
hidden node #8's value: 0.8151170454839437
hidden node #9's value: 0.005759852918451121
hidden node #10's value: 0.372578881044655
hidden node #11's value: 0.999678811623264
[0.14736157947701156, 0.8526384205229884]
Genome takes action 1
hidden node #8's value: 0.8579486685638456
hidd

...

hidden node #8's value: 0.5322933965495052
hidden node #9's value: 0.00011048657146980698
hidden node #10's value: 0.00011048657146980698
hidden node #11's value: 0.00010205536312395896
[0.2788538248311204, 0.7211461751688796]
Genome takes action 1
hidden node #8's value: 0.5301986002790579
hidden node #9's value: 2.202060072478521e-05
hidden node #10's value: 2.202060072478521e-05
hidden node #11's value: 1.9860527530547267e-05
[0.27639928168514255, 0.7236007183148575]
Genome takes action 1
hidden node #8's value: 0.5213206921888063
hidden node #9's value: 4.193762589934475e-06
hidden node #10's value: 4.193762589934475e-06
hidden node #11's value: 3.6790970075864194e-06
[0.273928270438532, 0.726071729561468]
Genome takes action 1
Episode finished after 9 timesteps
Genome fitness: 9.0
added hidden node
added hidden node
added hidden node
added hidden node
Connection already exists. Did not insert
Connection already exists. Did not insert
Connection already exists. Did not insert
Conne

hidden node #8's value: 0.9974919355412876
hidden node #9's value: 0.5494443500866655
hidden node #10's value: 0.9988261655433869
hidden node #11's value: 0.9926084586557181
[0.07457534580563756, 0.9254246541943625]
Genome takes action 1
hidden node #8's value: 0.9990630212521575
hidden node #9's value: 0.5688274654855082
hidden node #10's value: 0.9988830435899358
hidden node #11's value: 0.9926084586557181
[0.07119552651432146, 0.9288044734856786]
Genome takes action 1
hidden node #8's value: 0.999652968339376
hidden node #9's value: 0.5926401617478012
hidden node #10's value: 0.9989485488733632
hidden node #11's value: 0.9926084586557181
[0.0649503628623062, 0.9350496371376937]
Genome takes action 1
hidden node #8's value: 0.9998724828040279
hidden node #9's value: 0.6205565230551937
hidden node #10's value: 0.9990196297910447
hidden node #11's value: 0.9926084586557181
[0.056074520389650995, 0.943925479610349]
Genome takes action 1
hidden node #8's value: 0.9999534845771595
hidden 

hidden node #8's value: 0.25342530417642933
hidden node #9's value: 0.9926084586557181
hidden node #10's value: 0.9309322818797413
hidden node #11's value: 0.9457735851800758
[0.4315433583863295, 0.5684566416136706]
Genome takes action 1
hidden node #8's value: 0.3394018573024668
hidden node #9's value: 0.9926084586557181
hidden node #10's value: 0.9535751872756917
hidden node #11's value: 0.8203778714345105
[0.5222802687565187, 0.47771973124348144]
Genome takes action 0
hidden node #8's value: 0.24092095682274958
hidden node #9's value: 0.9926084586557181
hidden node #10's value: 0.9268871572380842
hidden node #11's value: 0.9542999009206954
[0.42786973441482645, 0.5721302655851737]
Genome takes action 1
hidden node #8's value: 0.3243489203752498
hidden node #9's value: 0.9926084586557181
hidden node #10's value: 0.9501984529439832
hidden node #11's value: 0.8490991732667037
[0.513286509326817, 0.4867134906731831]
Genome takes action 0
hidden node #8's value: 0.22861189648025523
hidde

hidden node #11's value: 0.999957240413668
[0.11446177003338595, 0.8855382299666141]
Genome takes action 1
hidden node #8's value: 0.5097581571143635
hidden node #9's value: 0.9248439581243497
hidden node #10's value: 0.9981625404964166
hidden node #11's value: 0.9999821721045861
[0.11232552947644299, 0.887674470523557]
Genome takes action 1
hidden node #8's value: 0.5285923975519861
hidden node #9's value: 0.9325170347138982
hidden node #10's value: 0.9989170436290092
hidden node #11's value: 0.9999926258072884
[0.10994303255684866, 0.8900569674431513]
Genome takes action 1
hidden node #8's value: 0.552081945629961
hidden node #9's value: 0.9410761571671677
hidden node #10's value: 0.9993731708795304
hidden node #11's value: 0.9999969664250019
[0.10739392149958862, 0.8926060785004113]
Genome takes action 1
hidden node #8's value: 0.5800157310634684
hidden node #9's value: 0.9499473131382059
hidden node #10's value: 0.9996426463755868
hidden node #11's value: 0.9999987553433214
[0.1047

hidden node #8's value: 0.42637524328917653
hidden node #9's value: 0.024482886926124665
hidden node #10's value: 0.9926084586557181
hidden node #11's value: 0.0021681520449665114
[0.09588740106111315, 0.9041125989388868]
Genome takes action 1
hidden node #8's value: 0.4077359847124756
hidden node #9's value: 0.009956459351970174
hidden node #10's value: 0.9926084586557181
hidden node #11's value: 0.00046772932066145193
[0.075756558589914, 0.924243441410086]
Genome takes action 1
hidden node #8's value: 0.38494602356989116
hidden node #9's value: 0.003779187952250368
hidden node #10's value: 0.9926084586557181
hidden node #11's value: 9.755356699308595e-05
[0.05883835056277485, 0.9411616494372252]
Genome takes action 1
hidden node #8's value: 0.3582080328215398
hidden node #9's value: 0.0013355598235152458
hidden node #10's value: 0.9926084586557181
hidden node #11's value: 1.9564291080907168e-05
[0.04502087706760796, 0.9549791229323921]
Genome takes action 1
Episode finished after 8 t

In [ ]:
# make sure genome 1 is the higher fitness genome 
def crossover(genome1, genome2):
    if (genome2.)

In [ ]:
genome.printNodeGenes()
genome.printActiveConnectGenes()

In [ ]:
# TODO: Verify this works properly
def delta(genome1, genome2, c1=0.1, c2=0.1, c3=0.1):
    if (not genome1.connectGenes or not genome2.connectGenes):
        return 0
    connectionMismatch = []
    for connection1 in genome1.connectGenes:
        if (connection1 not in genome2.connectGenes):
            connectionMismatch.append(connection1)
    for connection2 in genome2.connectGenes:
        if (connection2 not in genome1.connectGenes):
            connectionMismatch.append(connection2)
    diffsum = 0
    num = 0
    for connection1 in genome1.connectGenes:
        if (connection1 not in connectionMismatch):
            diffsum += abs(genome1.connectGenes[connection1][1] -
                           genome2.connectGenes[connection1][1])
            num += 1
    averageWeightDifference = diffsum / num
    #print(averageWeightDifference)
    N = 0
    if len(genome1.connectGenes) >= len(genome2.connectGenes):
        N = len(genome1.connectGenes)
    else:
        N = len(genome2.connectGenes)
    #print(((len(connectionMismatch)*(c1 + c2))/N) + (c3*averageWeightDifference))
    return ((len(connectionMismatch) *
             (c1 + c2)) / N) + (c3 * averageWeightDifference)

In [ ]:
# connection weights mutate as in any NE system, with each connection either perturbed or not at each generation 

In [ ]:
maxNodes = 100000
max_generations = 150
population = 150

globalInnovation = 1

inputs = env.observation_space.shape[0] + 1
outputs = env.action_space.n
# TODO: consider making these into dictionaries ?
for generation in range(max_generations):
    print('Starting new generation: {}'.format(generation))
    species = {}
    speciesCount = 0
    pop = {}
    for genome in range(population):
        # i think what I'll attempt first is 
        # create the genome, then subject it to all of the random chances 
        g = Genome()
        # if this is the first generation, then just initRandomGenome
        if (generation == 1):
            g.initRandomGenome(inputs, outputs)
        else: # else, initialize the genome from something else (?) have trouble with this in terms of cross over and stuff
            # TODO: replace this 
            g.initRandomGenome(inputs, outputs)
        # insert all of the random chances here 
        if (g.connectGenes): # if there are any connections
            if (random.random() <= genomeMutationChance):
                for connection in g.connectGenes:
                    if (random.random() <= connectionUniformPerturb): # uniformly perturb the connection
                        g.connectGenes[connection][1] += random.uniform(-perturbVal, perturbVal)
                    else:
                        g.connectGenes[connection][1] = random.random() # set a new value for 
            
                
            
        pop[genome] = g # add the genome into the current population 
        
        if (not species): # if there are no species in the species list, initialize it with the first genome created
            print('Species list empty. Creating new species')
            species[speciesCount] = [g]
            speciesCount += 1
        else:
            for og in species: # for all of the species in the species list 
                if (delta(g, species[og][0], c1, c2, c3) > deltaThreshold): # species[og][0] gets the first genome of a species
                    print('DeltaThreshold surpassed. Creating new species')
                    species[speciesCount] = g 
                    speciesCount += 1 
                else:
                    print('Adding genome into species#{}'.format(og))
                    species[og].append(g)
                    g.species = og
                    break
                    
    print("Number of different species in generation {}: {}".format(generation, len(species)))

# for generation in range(max_generations):
#     species = []
#     pop= []
#     for genome in range(population):
#         g = Genome()
#         g.initRandomGenome()
#         pop.append(g)
        
#         if (not species):
#             species.append(g)
#         else:
#             for og in species:
#                 if (delta(g, og, c1, c2, c3) > deltaThreshold):
#                     species.append(g)
#                 else:
#                     g.species = species.index(og)

                    
                    
#     print("Number of different species in generation {}: {}".format(generation, len(species)))


In [ ]:
# testing it to run in gym 

for tries in range(20):
    observation = env.reset()
    genome=Genome()
    genome.initRandomGenome()
    t = 0
    while(True):
        env.render()
        print(observation)
        action = genome.evaluateAction(observation)
        observation, reward, done, info = env.step(action)
        genome.fitness += reward
        if done:
            print("Episode finished after {} timesteps".format(t+1))
            print("Genome fitness: {}".format(genome.fitness))
            break
        t += 1
            
env.close()

# for i_episode in range(20):
#     observation = env.reset()
#     for t in range(100):
#         env.render()
#         print(observation)
#         action = env.action_space.sample()
#         observation, reward, done, info = env.step(action)
#         if done:
#             print("Episode finished after {} timesteps".format(t+1))
#             break
# env.close()